# Opérations sur les vecteurs

In [1]:
import numpy as np
import gensim, logging
import operator
import scipy
from scipy.spatial.distance import squareform, pdist, cosine
import os
import xml.etree.ElementTree as et
import pandas as pd
import networkx as nx
import seaborn as sns
sns.set(style="whitegrid")


In [2]:
%store -r overabundance dfVerbsBNC dfTripletsBNC dfHomographesBNC dfHeterographesBNC dfHomographes dfHeterographes

In [3]:
freqMin=50
len(dfHomographesBNC)

199

In [4]:
# load and use Word2Vec for cosine similarities
modelRaw = gensim.models.Word2Vec.load('word2vec_model_100n_spw_homog2_raw')

In [5]:
# load and use Word2Vec for cosine similarities
modelTag = gensim.models.Word2Vec.load('word2vec_model_100n_spw_homog2')

In [6]:
modelTag.wv.vocab

{'welcome': <gensim.models.keyedvectors.Vocab at 0x7fad61e11a60>,
 'to': <gensim.models.keyedvectors.Vocab at 0x7fad676bae80>,
 'somerset': <gensim.models.keyedvectors.Vocab at 0x7fad61daf550>,
 'so': <gensim.models.keyedvectors.Vocab at 0x7fad61e09d00>,
 'easy': <gensim.models.keyedvectors.Vocab at 0x7fad61e09280>,
 'get_base': <gensim.models.keyedvectors.Vocab at 0x7fad61d7b700>,
 'by': <gensim.models.keyedvectors.Vocab at 0x7fad61d7b6a0>,
 'car_n': <gensim.models.keyedvectors.Vocab at 0x7fad69b566a0>,
 'from': <gensim.models.keyedvectors.Vocab at 0x7fad69b56940>,
 'the': <gensim.models.keyedvectors.Vocab at 0x7fad69b567f0>,
 'north_n': <gensim.models.keyedvectors.Vocab at 0x7fad69b56a60>,
 'and': <gensim.models.keyedvectors.Vocab at 0x7fad69b56ac0>,
 'midlands': <gensim.models.keyedvectors.Vocab at 0x7fad69b56b20>,
 'down': <gensim.models.keyedvectors.Vocab at 0x7fad69b56b80>,
 'motorway_n': <gensim.models.keyedvectors.Vocab at 0x7fad69b56be0>,
 '21': <gensim.models.keyedvectors.Voc

In [7]:
modelRaw.wv.most_similar(positive=['china'], negative=[], topn=10)

[('brazil', 0.7308750748634338),
 ('india', 0.6943464279174805),
 ('canada', 0.6743540167808533),
 ('asia', 0.662615954875946),
 ('greece', 0.6467833518981934),
 ('sweden', 0.6336503028869629),
 ('america', 0.632462739944458),
 ('australia', 0.6295404434204102),
 ('switzerland', 0.6186487078666687),
 ('italy', 0.6089438199996948)]

In [6]:
# modelTag.wv.most_similar(positive=['remember_base', 'went_past'], negative=['go_base'], topn=10)

### Calcul de la matrice des vecteurs de toutes les formes verbales au dessus du seuil de fréquence

In [7]:
model=modelRaw
lForms=list(set(dfVerbsBNC[dfVerbsBNC.oPST>=freqMin].PST.unique().tolist()+dfVerbsBNC[dfVerbsBNC.oPART>=freqMin].PART.unique().tolist()))
sForms=pd.Series([model.wv[v] for v in lForms],index=lForms)
sForms
dfForms=pd.DataFrame(sForms.values.tolist(), index=sForms.index)
dfForms

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
drove,-0.191994,0.154058,-0.167808,-1.375660,0.454771,0.458397,-0.559237,-0.388561,0.906991,-1.338941,...,-0.452810,0.361990,0.130091,0.527031,-0.426289,0.365022,-0.934915,-0.472189,1.030952,2.369463
granted,0.495698,-1.037804,1.320854,-1.063405,-0.689632,-2.315900,-0.241521,0.365219,0.925270,-1.574150,...,-0.225016,-1.981418,0.774670,0.423046,-0.066557,-0.905936,0.332578,0.510632,-1.824590,0.708725
bound,-0.117439,0.323277,-0.219055,-1.006207,-0.200617,-0.863110,1.349070,-0.300756,0.674548,0.443009,...,-0.982544,-0.016043,-1.114756,-0.661707,-0.566460,0.439738,-1.724078,0.896062,-1.401108,0.967868
produced,0.858515,0.939373,0.269545,-0.745994,-0.306690,0.129255,1.358666,0.117649,0.710154,-0.755120,...,0.573716,-0.673941,-0.584177,1.453884,-0.333814,0.542534,0.195713,0.894406,-1.140965,1.167620
formed,-0.845062,0.494577,1.827999,-2.285610,-1.029777,-0.237229,-0.163557,-0.525746,1.451937,0.307241,...,0.187935,-1.214642,1.497267,0.623418,1.797004,0.129330,-0.253330,0.122959,-2.431916,1.808626
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
declined,-0.222045,-0.013028,0.023469,0.274090,0.303352,-1.771279,0.518280,-1.029083,1.133157,-0.611210,...,0.135399,-1.042916,1.327363,-0.038491,2.007625,0.000982,0.240299,0.230912,-1.283446,2.000640
fought,-0.195650,-0.293592,0.296481,-0.362252,0.021644,-0.737526,0.295277,0.042326,0.976167,-0.519443,...,-0.433469,-0.798770,0.702793,-0.226520,1.011158,-0.476590,-0.088091,0.490389,0.280342,1.344519
realised,-1.663822,-1.211019,0.501054,-1.509193,-0.117366,-1.795740,-0.458752,-2.395461,1.170929,-1.379824,...,-0.896048,-0.036983,0.106636,0.744161,-1.256426,-0.508830,-0.071293,0.034035,-0.500745,1.751649
recorded,1.053239,1.080721,-0.031049,1.072190,0.093468,-0.394544,-1.367281,-0.723332,1.284305,0.222657,...,-1.015455,-0.521559,0.521907,-0.274395,1.130104,-0.726735,1.163788,1.881689,-1.202559,2.328246


In [8]:
sHomographesCandidates=pd.Series([model.wv[v] for v in dfHomographesBNC.index],index=dfHomographesBNC.index)
dfHomographesCandidates=pd.DataFrame(sHomographesCandidates.values.tolist(), index=sHomographesCandidates.index)
dfHomographesCandidates

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
have,0.761518,2.487359,0.768001,-2.424294,2.097283,-0.012102,-0.032491,0.061929,-2.034181,0.945525,...,0.568268,-3.023235,-2.896835,-0.933884,0.050817,2.082963,-1.111177,1.240312,2.600351,-1.324106
make,3.140661,3.496083,0.649879,0.854973,1.508227,-3.667590,1.054231,-3.826508,-0.965403,0.382941,...,0.018452,-0.243939,-2.061839,1.637392,-1.031885,3.810454,-1.113531,-0.227947,-0.167529,-1.525081
find,0.936304,1.269175,1.309222,0.290079,1.750264,-3.536414,-1.791553,-3.498623,-2.336558,1.372058,...,0.892447,-0.472111,-0.339450,1.913985,-0.580115,0.803082,-1.224129,-0.797848,2.512517,-0.858219
say,-0.818348,-0.008638,0.056245,-0.782897,0.166533,-3.347827,0.739416,-3.262314,0.673314,-1.637403,...,0.444943,0.061952,-1.475002,2.383390,0.690439,0.812115,-2.172788,0.229276,1.324040,0.684298
think,-1.092052,0.398231,-0.205900,-0.081753,0.474781,-4.165641,-0.321932,-2.178291,0.274254,0.392870,...,-1.305050,-0.256155,-0.714936,2.167286,-0.281732,0.874815,-0.414261,-1.237652,1.151801,0.677480
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
promise,-1.026089,-0.826553,0.531242,-0.195943,-0.034208,-0.479046,0.970869,-0.637082,-0.219090,-0.926630,...,-0.107396,0.429024,-0.850600,0.134754,-0.547375,0.983841,0.750412,0.100502,1.507632,-0.413499
publish,0.252796,-0.034514,-0.527871,0.777598,0.494348,-0.022987,-0.300315,-0.866796,-0.356472,0.335150,...,-0.000537,0.924206,-0.221328,-0.632390,-0.822608,0.412218,-0.469265,-0.090258,-0.386567,0.330360
warn,-0.242187,-0.148811,0.150881,-0.148764,0.343642,-0.773164,-0.244045,-1.076447,-0.390426,-0.028393,...,0.132368,-0.024745,0.136017,-0.439630,-0.476547,-0.208789,-0.413366,-0.240183,0.405735,0.869086
name,1.860188,-1.671901,1.502887,-1.087506,0.803158,0.544991,0.903952,0.571978,-1.571505,-1.791369,...,-1.201898,-0.247137,-0.797957,0.833416,-0.705321,-1.190250,0.704699,-1.293130,0.725652,-0.230812


### Calcul des vecteurs des homographes

In [9]:
sHomographesCandidates=pd.Series([model.wv[v] for v in dfHomographesBNC.index],index=dfHomographesBNC.index)
dfHomographesCandidates=pd.DataFrame(sHomographesCandidates.values.tolist(), index=sHomographesCandidates.index)
dfHomographesCandidates

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
have,0.761518,2.487359,0.768001,-2.424294,2.097283,-0.012102,-0.032491,0.061929,-2.034181,0.945525,...,0.568268,-3.023235,-2.896835,-0.933884,0.050817,2.082963,-1.111177,1.240312,2.600351,-1.324106
make,3.140661,3.496083,0.649879,0.854973,1.508227,-3.667590,1.054231,-3.826508,-0.965403,0.382941,...,0.018452,-0.243939,-2.061839,1.637392,-1.031885,3.810454,-1.113531,-0.227947,-0.167529,-1.525081
find,0.936304,1.269175,1.309222,0.290079,1.750264,-3.536414,-1.791553,-3.498623,-2.336558,1.372058,...,0.892447,-0.472111,-0.339450,1.913985,-0.580115,0.803082,-1.224129,-0.797848,2.512517,-0.858219
say,-0.818348,-0.008638,0.056245,-0.782897,0.166533,-3.347827,0.739416,-3.262314,0.673314,-1.637403,...,0.444943,0.061952,-1.475002,2.383390,0.690439,0.812115,-2.172788,0.229276,1.324040,0.684298
think,-1.092052,0.398231,-0.205900,-0.081753,0.474781,-4.165641,-0.321932,-2.178291,0.274254,0.392870,...,-1.305050,-0.256155,-0.714936,2.167286,-0.281732,0.874815,-0.414261,-1.237652,1.151801,0.677480
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
promise,-1.026089,-0.826553,0.531242,-0.195943,-0.034208,-0.479046,0.970869,-0.637082,-0.219090,-0.926630,...,-0.107396,0.429024,-0.850600,0.134754,-0.547375,0.983841,0.750412,0.100502,1.507632,-0.413499
publish,0.252796,-0.034514,-0.527871,0.777598,0.494348,-0.022987,-0.300315,-0.866796,-0.356472,0.335150,...,-0.000537,0.924206,-0.221328,-0.632390,-0.822608,0.412218,-0.469265,-0.090258,-0.386567,0.330360
warn,-0.242187,-0.148811,0.150881,-0.148764,0.343642,-0.773164,-0.244045,-1.076447,-0.390426,-0.028393,...,0.132368,-0.024745,0.136017,-0.439630,-0.476547,-0.208789,-0.413366,-0.240183,0.405735,0.869086
name,1.860188,-1.671901,1.502887,-1.087506,0.803158,0.544991,0.903952,0.571978,-1.571505,-1.791369,...,-1.201898,-0.247137,-0.797957,0.833416,-0.705321,-1.190250,0.704699,-1.293130,0.725652,-0.230812


# Introduire les vecteurs somme pour les hétérographes pour Raw

In [10]:
lCOMB=[]
for ix,v in dfHeterographesBNC.iterrows():
    vPST=v["PST"]
    vPART=v["PART"]
    vSum=modelRaw.wv[vPST]+modelRaw.wv[vPART]
    vCOMB=vPST+"+"+vPART
    print(vCOMB)
    modelRaw.wv.add(vCOMB,vSum)
    lCOMB.append(vCOMB)
modelRaw.wv.vectors_norm = None

did+done
ate+eaten
spoke+spoken
grew+grown
drew+drawn
forgot+forgotten
drove+driven
threw+thrown
blew+blown
sank+sunk


In [11]:
dNeighbours={}
dNeighbourCandidates=dfHeterographesBNC["PST"].tolist()+dfHeterographesBNC["PST"].tolist()+lCOMB
dNeighbourCandidates

['did',
 'ate',
 'spoke',
 'grew',
 'drew',
 'forgot',
 'drove',
 'threw',
 'blew',
 'sank',
 'did',
 'ate',
 'spoke',
 'grew',
 'drew',
 'forgot',
 'drove',
 'threw',
 'blew',
 'sank',
 'did+done',
 'ate+eaten',
 'spoke+spoken',
 'grew+grown',
 'drew+drawn',
 'forgot+forgotten',
 'drove+driven',
 'threw+thrown',
 'blew+blown',
 'sank+sunk']

### Calcul de la matrice des vecteurs pour les formes de référence y compris les combinaisons

In [12]:
sNeighbourCandidates=pd.Series([model.wv[v] for v in dNeighbourCandidates],index=dNeighbourCandidates)
dfNeighbourCandidates=pd.DataFrame(sNeighbourCandidates.values.tolist(), index=sNeighbourCandidates.index)
dfNeighbourCandidates

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
did,-1.782079,0.505477,0.171306,-0.444704,-0.263907,-2.937971,1.372542,-1.244532,-0.859461,-2.221965,...,-1.368545,1.047679,-1.515349,-0.585773,2.737542,-0.371525,-0.062150,-1.901034,3.514918,0.679551
ate,-0.320411,-0.127873,0.061216,-0.394844,0.016624,-0.166972,-0.035922,-0.843693,0.514115,-0.200611,...,-0.710716,0.115886,0.121392,0.843341,-0.008031,0.379093,-0.059419,0.242928,0.062918,0.483665
spoke,-0.287752,-1.314700,-0.984803,-1.698618,0.604435,-0.187868,-1.952597,-1.284732,1.706807,-1.337715,...,-1.241809,0.763108,0.266332,0.533654,-0.048548,-0.704632,0.081576,0.322706,1.417498,2.749679
grew,-0.103036,0.379452,-0.298987,-0.444171,-0.462859,-0.172357,0.710718,-0.655705,1.253052,-0.837803,...,0.888950,0.021910,0.488760,0.704361,0.194398,-0.040221,1.342786,-0.313025,0.985941,2.409637
drew,-0.239078,-1.057162,-0.110416,-0.076608,0.037862,1.252831,-0.949254,0.325087,1.524888,-1.094705,...,-1.169670,1.236152,0.589721,0.182162,-0.723282,0.744422,-0.481673,-0.314105,0.718047,1.953690
forgot,-0.665443,-0.509979,-0.008594,-0.870253,0.466324,-0.194436,-0.518253,-0.452313,0.734129,-0.316272,...,-0.663842,0.753901,-0.088465,-0.028817,0.088741,-0.516097,0.124028,0.072849,0.396442,0.447612
drove,-0.191994,0.154058,-0.167808,-1.375660,0.454771,0.458397,-0.559237,-0.388561,0.906991,-1.338941,...,-0.452810,0.361990,0.130091,0.527031,-0.426289,0.365022,-0.934915,-0.472189,1.030952,2.369463
threw,-0.048719,-0.236378,-0.833735,-0.409387,0.856810,0.468943,-0.012812,0.850710,1.284015,-0.941811,...,-1.197978,0.482205,-0.324657,0.303264,-0.822486,0.663176,0.000489,0.153921,0.733996,1.680736
blew,0.188131,0.170785,-0.438014,-0.841930,-0.112844,0.690787,-0.271922,1.181479,0.234745,-0.703082,...,-0.365755,0.434714,-0.133726,0.096172,-0.880874,-0.108300,0.143554,-0.331873,0.400960,0.418539
sank,-0.527877,0.323275,-0.204352,-0.726638,0.133805,0.386780,0.291098,0.682358,1.119720,-0.182881,...,-0.212356,0.127712,0.387611,0.447735,0.303872,0.259488,-0.314293,0.242717,0.382224,0.927673


# Calcul du vecteur de référence le plus proche

## pour les homographes

In [13]:
for ix,v in dfHomographesBNC.iterrows():
    vCOMB=v["PART"]
    neighbours=modelRaw.wv.most_similar(vCOMB, topn=1000)
    neighboursCOMB=[(i,v) for i,(v,s) in enumerate(neighbours) if v in dNeighbourCandidates]
    if neighboursCOMB:
#        print(vCOMB,neighboursCOMB[0])
        dNeighbours[vCOMB]=neighboursCOMB[0]
    else:
#        print(vCOMB,"pas de voisin combiné")
        dNeighbours[vCOMB]=(None,None)
    print
dfNeighbours=pd.DataFrame(dNeighbours).T
dfNeighbours.columns="rank distribution".split(" ")
dfNeighbours["rank"]=dfNeighbours["rank"].astype(str)
dfNeighbourhoods=dfNeighbours.reset_index().groupby("distribution").agg({"index":", ".join,"rank":[", ".join,"count"]})
dfNeighbourhoods.columns=["syncretic forms","neighbouring rank","number"]
dfNeighbourhoods

,syncretic forms,neighbouring rank,number
distribution,,,
ate+eaten,"found, used, provided, included, produced, bou...","51, 242, 682, 841, 287, 8, 156, 79, 98, 248, 2...",26
blew,opened,38,1
blew+blown,shot,2,1
did,"wanted, let, refused","10, 68, 90",3
did+done,"needed, helped, tried, meant, understood, paid...","60, 108, 20, 6, 65, 235, 53, 45, 191, 7, 282, ...",14
drew+drawn,"made, brought, set, held, obtained, sought, in...","32, 1, 11, 66, 222, 229, 607, 104, 4, 88, 406,...",21
drove,"started, stopped, stayed, returned, watched, hit","42, 15, 55, 35, 11, 7",6
drove+driven,"moved, followed, offered, called, built, creat...","7, 16, 436, 24, 30, 529, 99, 616, 23, 378, 10,...",21
forgot,"had, said, felt, talked","18, 43, 11, 2",4


## Calcul du vecteur de référence le plus proche pour tous les vecteurs

In [14]:
dNeighbourCandidates

['did',
 'ate',
 'spoke',
 'grew',
 'drew',
 'forgot',
 'drove',
 'threw',
 'blew',
 'sank',
 'did',
 'ate',
 'spoke',
 'grew',
 'drew',
 'forgot',
 'drove',
 'threw',
 'blew',
 'sank',
 'did+done',
 'ate+eaten',
 'spoke+spoken',
 'grew+grown',
 'drew+drawn',
 'forgot+forgotten',
 'drove+driven',
 'threw+thrown',
 'blew+blown',
 'sank+sunk']

In [15]:
dFormNeighbours={}
for ix,v in dfForms.iterrows():
    vCOMB=ix
    formNeighbours=modelRaw.wv.most_similar(vCOMB, topn=1000)
    formNeighboursCOMB=[(i,v[0]) for i,v in enumerate(formNeighbours) if v[0] in dNeighbourCandidates]
    if formNeighboursCOMB:
        dFormNeighbours[vCOMB]=formNeighboursCOMB[0]
    else:
#        print(vCOMB,"pas de voisin combiné")
        dFormNeighbours[vCOMB]=(None,None)
    print
dfFormNeighbours=pd.DataFrame(dFormNeighbours).T
#print(dfFormNeighbours)
dfFormNeighbours.columns="rank distribution".split(" ")
dfFormNeighbours["rank"]=dfFormNeighbours["rank"].astype(str)
dfFormNeighbourhoods=dfFormNeighbours.reset_index().groupby("distribution").agg({"index":", ".join,"rank":[", ".join,"count"]})
dfFormNeighbourhoods.columns=["syncretic forms","neighbouring rank","number"]
dfFormNeighbourhoods

,syncretic forms,neighbouring rank,number
distribution,,,
ate,slept,4,1
ate+eaten,"produced, adjusted, encountered, included, sto...","287, 490, 248, 841, 286, 255, 325, 531, 155, 5...",63
blew,"burst, woke, rang, opened","0, 6, 3, 38",4
blew+blown,"washed, painted, shot, rolled, blew, blown, sp...","2, 260, 2, 0, 0, 0, 18, 6, 14, 32",10
did,"lacked, refused, wanted, let","376, 90, 10, 68",4
did+done,"sustained, achieved, planned, suited, dismisse...","874, 256, 753, 563, 318, 60, 463, 125, 45, 207...",27
drew,"arose, enabled, struggled","436, 263, 21",3
drew+drawn,"bound, completed, viewed, brought, executed, i...","39, 477, 383, 1, 379, 553, 11, 41, 294, 517, 8...",48
drove,"returned, marched, waited, hurried, travelled,...","35, 5, 22, 17, 20, 9, 136, 10, 790, 42, 55, 11...",15


In [16]:
dfVerbsBNC[dfVerbsBNC["PST"].isin(["looked","stood"])]

Hétérographes,BASE,PST,PART,oBASE,oPST,oPART
look,look,looked,looked,1873,1423,208
stand,stand,stood,stood,511,749,65


- 27 modèles simples
- 162 modèles composés

In [17]:
dfNeighbourhoods[dfNeighbourhoods.index.str.contains("\+")].sum()

syncretic forms      found, used, provided, included, produced, bou...
neighbouring rank    51, 242, 682, 841, 287, 8, 156, 79, 98, 248, 2...
number                                                             162
dtype: object

In [ ]:
# dfVerbsBNC.loc[dfVerbsBNC.index.isin(["drive"]+"start stop stay return watch hit".split(" "))]

In [ ]:
# dfVerbsBNC.loc[dfVerbsBNC.index.isin(["throw","look","keep","put","carry"])]

# Précision des prédictions de PAST vers PART

In [ ]:
dPrecisionPast={}

In [ ]:
goodVectors=[]
badVectors=[]
totalVectors=[]

nbTop=3

for ix,v in dfNeighbourhoods.iterrows():
    if "+" in ix:
        falsePositives=[]
        truePositives=[]
        pPast,pPart=ix.split("+")
        combForms=v["index"].values[0].split(", ")
        combRangs=v["rang"].values[0].split(", ")
        for nForm,form in enumerate(combForms):
#            print(combRangs[nForm],form) 
            totalVectors.append(form)
            combVoisins=modelTag.wv.most_similar(positive=["%s_past"%form,"%s_part"%pPart],negative=["%s_past"%pPast],topn=nbTop)
            falsePositive=True
            for n in range(nbTop):
                if combVoisins[n][0]=="%s_part"%form:
                    truePositives.append((form,n,combRangs[nForm]))
                    goodVectors.append(form)
                    falsePositive=False
                    break
            if falsePositive:
                falsePositives.append((form,None,combRangs[nForm]))
                badVectors.append(form)
        combPrecision=float(len(truePositives))/len(combForms)        
        print(ix,combPrecision)
        if truePositives:
            print()
            print("True",truePositives)
        if falsePositives:
            print()
            print("False",falsePositives)
        print()
        print()

In [ ]:
precision=float(len(goodVectors))/len(totalVectors)
print(nbTop,precision)
dPrecisionPast[nbTop]=precision

print(dPrecisionPast)
print()
print(badVectors)

In [ ]:
dfVerbsBNC[dfVerbsBNC["PST"].isin(badVectors)]

In [ ]:
dfTopnPrecisionPastPart=pd.DataFrame(dPrecisionPast.items())
dfTopnPrecisionPastPart.columns=["neighbourhood size", "precision"]
dfGraph=dfTopnPrecisionPastPart.sort_values("neighbourhood size").set_index("neighbourhood size")
dfGraph.plot(xticks=dfGraph.index,yticks=[0,0.2,0.4,0.6,0.8,1])

# Précision des prédictions de BASE vers PART

In [ ]:
dPrecisionBase={}

In [ ]:
goodVectors=[]
badVectors=[]
totalVectors=[]

nbTop=18

for ix,v in dfNeighbourhoods.iterrows():
    if "+" in ix:
        falsePositives=[]
        truePositives=[]
        pPast,pPart=ix.split("+")
        pBase=dfVerbsBNC[dfVerbsBNC["PST"]==pPast]["BASE"].values[0]
        combForms=v["index"].values[0].split(", ")
        combRangs=v["rang"].values[0].split(", ")
        for nForm,form in enumerate(combForms):
            base=dfVerbsBNC[dfVerbsBNC["PST"]==form]["BASE"].values[0]
            totalVectors.append(form)
            combVoisins=modelTag.wv.most_similar(positive=["%s_base"%base,"%s_part"%pPart],negative=["%s_base"%pBase],topn=nbTop)
            falsePositive=True
            for n in range(nbTop):
                if combVoisins[n][0]=="%s_part"%form:
                    truePositives.append((form,n,combRangs[nForm]))
                    goodVectors.append(form)
                    falsePositive=False
                    break
            if falsePositive:
                falsePositives.append((form,None,combRangs[nForm]))
                badVectors.append(form)
        combPrecision=float(len(truePositives))/len(combForms)        
        print(ix,combPrecision)
        if truePositives:
            print()
            print("True",truePositives)
        if falsePositives:
            print()
            print("False",falsePositives)
        print()
        print()

In [ ]:
precision=float(len(goodVectors))/len(totalVectors)
print(nbTop,precision)
dPrecisionBase[nbTop]=precision

print(dPrecisionBase)
print()
print(badVectors)

In [ ]:
dfVerbsBNC[dfVerbsBNC["PST"].isin(badVectors)]

In [ ]:
dfTopnPrecisionBasePart=pd.DataFrame(dPrecisionBase.items())
dfTopnPrecisionBasePart.columns="topn precision".split(" ")
dfGraph=dfTopnPrecisionBasePart.sort_values("topn").set_index("topn")
dfGraph.plot()

# Précision des prédictions de BASE vers PAST

In [ ]:
dPrecisionBasePast={}

In [ ]:
goodVectors=[]
badVectors=[]
totalVectors=[]

nbTop=20

for ix,v in dfNeighbourhoods.iterrows():
    if "+" in ix:
        falsePositives=[]
        truePositives=[]
        pPast,pPart=ix.split("+")
        pBase=dfVerbsBNC[dfVerbsBNC["PST"]==pPast]["BASE"].values[0]
        combForms=v["index"].values[0].split(", ")
        combRangs=v["rang"].values[0].split(", ")
        for nForm,form in enumerate(combForms):
            base=dfVerbsBNC[dfVerbsBNC["PST"]==form]["BASE"].values[0]
            totalVectors.append(form)
            combVoisins=modelTag.wv.most_similar(positive=["%s_base"%base,"%s_past"%pPast],negative=["%s_base"%pBase],topn=nbTop)
            falsePositive=True
            for n in range(nbTop):
                if combVoisins[n][0]=="%s_past"%form:
                    truePositives.append((form,n,combRangs[nForm]))
                    goodVectors.append(form)
                    falsePositive=False
                    break
            if falsePositive:
                falsePositives.append((form,None,combRangs[nForm]))
                badVectors.append(form)
        combPrecision=float(len(truePositives))/len(combForms)        
        print(ix,combPrecision)
        if truePositives:
            print()
            print("True",truePositives)
        if falsePositives:
            print()
            print("False",falsePositives)
        print()
        print()

In [ ]:
precision=float(len(goodVectors))/len(totalVectors)
print(nbTop,precision)
dPrecisionBasePast[nbTop]=precision

print(dPrecisionBasePast)
print()
print(badVectors)

In [ ]:
dfVerbsBNC[dfVerbsBNC["PST"].isin(badVectors)]

In [ ]:
dfTopnPrecisionBasePast=pd.DataFrame(dPrecisionBasePast.items())
dfTopnPrecisionBasePast.columns="topn precision".split(" ")
dfGraph=dfTopnPrecisionBasePast.sort_values("topn").set_index("topn")
dfGraph.plot()

# Exploration du modèle Tag

In [ ]:
modelTag.wv.most_similar(positive=['seen_part'], topn=10)

In [ ]:
dfHeteroTagBNC=dfHeterographesBNC.copy()
dfHeteroTagBNC["PST"]=dfHeteroTagBNC["PST"]+"_past"
dfHeteroTagBNC["PART"]=dfHeteroTagBNC["PART"]+"_part"

dfTripletsTagBNC=dfTripletsBNC.copy()
dfTripletsTagBNC["BASE"]=dfTripletsTagBNC["BASE"]+"_base"
dfTripletsTagBNC["PST"]=dfTripletsTagBNC["PST"]+"_past"
dfTripletsTagBNC["PART"]=dfTripletsTagBNC["PART"]+"_part"

# Voisinages entre formes de la même case

In [ ]:
def getNeighbours(x):
    neighbours=[n[0]for n in model.wv.most_similar(x,topn=3)]
    return [x]+neighbours

In [ ]:
model=modelTag
dfTripletsTagBNC["BASE"].apply(getNeighbours)

In [ ]:
dfTripletsTagBNC["PST"].apply(getNeighbours)

In [ ]:
dfTripletsTagBNC["PART"].apply(getNeighbours)

In [ ]:
def getVect(x):
    return model.wv[x]

### Matrice des formes du prétérit

In [ ]:
model=modelTag

In [ ]:
sVectorsPast=dfTripletsTagBNC["PST"].apply(getVect)
dfVectorsPast=pd.DataFrame(sVectorsPast.values.tolist(), index=sVectorsPast.index)
dfPastDist=pd.DataFrame(squareform(pdist(dfVectorsPast,metric="cosine")), columns=dfVectorsPast.index.unique(), index=dfVectorsPast.index.unique())
dfPastDist

### Matrice des formes du participe

In [ ]:
sVectorsPart=dfTripletsTagBNC["PART"].apply(getVect)
dfVectorsPart=pd.DataFrame(sVectorsPart.values.tolist(), index=sVectorsPart.index)
dfPartDist=pd.DataFrame(squareform(pdist(dfVectorsPart,metric="cosine")), columns=dfVectorsPart.index.unique(), index=dfVectorsPart.index.unique())
dfPartDist

# Matrice de distance entre les vecteurs de translation (shift vectors)

## Calculer les vecteurs de translation pour les hétérographes

In [ ]:
def vectorTrans(row):
    w1=row["PART"]
    w2=row["PST"]
    result=model.wv[w1]-model.wv[w2]
    return result

In [ ]:
model=modelTag
sVectorsTag=dfHeteroBNC[["PST","PART"]].apply(vectorTrans, axis=1)
dfVectorsTag = pd.DataFrame(sVectorsTag.values.tolist(), index=sVectorsTag.index)
mVectorTag=dfVectorsTag.mean().to_list()
fVectorTag=dfVectorsTag.iloc[0]

In [ ]:
model=modelRaw
sVectorsRaw=dfHeterographesBNC[["PST","PART"]].apply(vectorTrans, axis=1)
dfVectorsRaw=pd.DataFrame(sVectorsRaw.values.tolist(), index=sVectorsRaw.index)
mVectorRaw=dfVectorsRaw.mean().to_list()
fVectorRaw=dfVectorsRaw.iloc[0]

In [ ]:
dfHeteroTagDist=pd.DataFrame(squareform(pdist(dfVectorsTag,metric="cosine")), columns=dfVectorsTag.index.unique(), index=dfVectorsTag.index.unique())
dfHeteroTagDist

In [ ]:
dfHeteroRawDist=pd.DataFrame(squareform(pdist(dfVectorsRaw,metric="cosine")), columns=dfVectorsRaw.index.unique(), index=dfVectorsRaw.index.unique())
dfHeteroRawDist

### Matrice de proximité entre vecteurs homographes et vecteurs combinés hétérographes

In [ ]:
model=modelRaw

In [ ]:
dfRawCombDist=pd.DataFrame(squareform(pdist(dfVectorsTag,metric="cosine")), columns=dfVectorsTag.index.unique(), index=dfVectorsTag.index.unique())
dfHeteroTagDist

In [ ]:

sVectorsRaw=pd.Series([model.wv[v] for v in dfHomographesBNC.PST.tolist()],index=dfHomographesBNC.PST.index)
dfVectorsRaw=pd.DataFrame(sVectorsRaw.values.tolist(), index=sVectorsRaw.index)
dfVectorsRaw
# sVectorsRaw

## Entraîner le modèle
- min_count : la fréquence minimale pour avoir un vecteur
- size : nombre de neurones en entrée

In [ ]:
minF=15
nNeurones=150
nIter=10

In [ ]:
#load and use Word2Vec for cosine similarities
model = gensim.models.Word2Vec.load("w2v-min%d-s%d-it%d"%(minF,nNeurones,nIter))

In [ ]:
def dist2mean(row):
    lVector=row.to_list()
    return cosine(lVector,mVector)

dfVectors.apply(dist2mean,axis=1)

In [ ]:
verb="go"
pivot="begin"
f1="PART"
f2="PST"
nVoisins=3
w1=dfHeterographes.loc[verb,f1]
w2=dfHeterographes.loc[verb,f2]
p1=dfHeterographes.loc[pivot,f1]
p2=dfHeterographes.loc[pivot,f2]
# model.wv.most_similar(positive=["gone","knew"],negative=["known"])
print(model.wv.most_similar(positive=[w1,p2],negative=[p1],topn=nVoisins))

vCalc=model.wv[w1]-model.wv[p1]+model.wv[p2]
vReal=model.wv[w2]
cosine(vReal,vCalc)

In [ ]:
def distCalcReal(verb,pivot):
    w1=dfHeterographes.loc[verb,f1]
    w2=dfHeterographes.loc[verb,f2]
    p1=dfHeterographes.loc[pivot,f1]
    p2=dfHeterographes.loc[pivot,f2]

    vCalc=model.wv[w1]-model.wv[p1]+model.wv[p2]
    vReal=model.wv[w2]
#     print(model.wv.rank(vCalc, vReal))
    neighbours=model.wv.most_similar(positive=[w1,p2],negative=[p1],topn=1000)
    for nN, (n,c) in enumerate(neighbours):
        if n==w2:
            #print(nN,n,c)
            break
    print(w1, w2, p1, p2, nN, c)

    return cosine(vReal,vCalc)

for v in dfHeterographes.index:
    for p in dfHeterographes.index:
        print (v, p, distCalcReal(v,p))

In [ ]:
len(list(model.wv.vocab.keys()))
# 1-cosine(model.wv[w1],model.wv[w2])